# BatchNorm1d example
In this example we show the `BatchNorm1d` layer in action.

We create a random input tensor and apply the `BatchNorm1d` layer to it. We then show that the mean and standard deviation of the output tensor are close to zero and one respectively.
We then create another input tensor with a different mean and standard deviation and show that the output tensor has a mean and standard deviation close to zero and one respectively.

We then switch the `BatchNorm1d` layer to *evaluation* mode and show that the output tensor has a mean and standard deviation close to the mean and standard deviation of the input tensor.

Finally we show that the output tensor has a mean and standard deviation close to the mean and standard deviation of the input tensor when the `BatchNorm1d` layer is in *evaluation* mode.

In [1]:
import torch
import torch.nn as nn

In [2]:
# BatchNorm 1d demo
# 1d input random uniform
x1 = torch.rand(10, 5)*4 - 3
print(x1)

tensor([[-0.4427, -1.2626,  0.8805, -1.7075,  0.0681],
        [-1.0230, -0.0345,  0.9077,  0.4888, -0.2472],
        [ 0.8176,  0.7956,  0.7026, -0.6514, -1.9642],
        [-1.9855, -1.0323, -2.1602,  0.5283,  0.8603],
        [-2.3401, -1.5615, -1.4883, -1.6763, -2.3242],
        [ 0.1865, -1.2540, -2.7504, -0.6765,  0.3262],
        [-0.4292, -1.8939, -1.0444, -0.0966, -1.2333],
        [ 0.8258, -2.3812, -0.5636, -0.8441, -2.9003],
        [-0.2809, -1.6568,  0.3018, -2.6939, -1.7417],
        [-2.3289, -1.0449, -1.6596, -0.9320,  0.8938]])


In [3]:
x1.mean(), x1.std()

(tensor(-0.8345), tensor(1.1430))

In [4]:
# Create batch norm layer. The input has 5 features.
# The batch norm layer will normalize each feature.
# The batch norm layer will learn the mean and standard deviation of each feature.
bn = nn.BatchNorm1d(5, affine=True, momentum=0.1)

In [5]:
# Apply batch norm layer to input
y1 = bn(x1)
# Show mean and standard deviation of output
y1.mean(), y1.std()

(tensor(-1.9073e-08, grad_fn=<MeanBackward0>),
 tensor(1.0101, grad_fn=<StdBackward0>))

In [6]:
# Create another input tensor with a different mean and standard deviation
x2 = torch.rand(10, 5) * 3.0 + 10.0
# Show mean and standard deviation of input
x2.mean(), x2.std()

(tensor(11.5915), tensor(0.9002))

In [7]:
# Apply batch norm layer to input
y2 = bn(x2)
# and show mean and standard deviation of output
y2.mean(), y2.std()

(tensor(1.3351e-07, grad_fn=<MeanBackward0>),
 tensor(1.0101, grad_fn=<StdBackward0>))

In [8]:
# Show the mode of the batch norm layer
bn.training

True

In [9]:
for p in bn.parameters():
	print(p)

Parameter containing:
tensor([1., 1., 1., 1., 1.], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)


In [10]:
# Show running mean and running variance of batch norm layer
bn.running_mean, bn.running_var

(tensor([1.1354, 1.0600, 1.0794, 1.0883, 1.0570]),
 tensor([1.0312, 0.9848, 1.0780, 0.9788, 1.0162]))

In [11]:
bn.num_batches_tracked

tensor(2)

In [12]:
bn.momentum

0.1

In [13]:
running_mean = 0.0
running_mean = bn.momentum * x1.mean(dim=0) + (1 - bn.momentum) * running_mean
running_mean = bn.momentum * x2.mean(dim=0) + (1 - bn.momentum) * running_mean
running_mean

tensor([1.1354, 1.0600, 1.0794, 1.0883, 1.0570])

# `BatchNorm1d` layer in *evaluation* mode

In [14]:
# Switch the batch norm layer to evaluation mode
bn.eval()


BatchNorm1d(5, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)

In [15]:
x3 = torch.rand(10, 5) * 8.0 + 20.0
x3.mean(), x3.std()

(tensor(24.1333), tensor(2.5012))

In [16]:
y3 = bn(x3)
y3.mean(), y3.std()

(tensor(22.8590, grad_fn=<MeanBackward0>),
 tensor(2.5341, grad_fn=<StdBackward0>))

In [17]:
_y3 = (x3 - bn.running_mean)/torch.sqrt(bn.running_var + bn.eps)
_y3.mean(), _y3.std()

(tensor(22.8590), tensor(2.5341))

In [18]:
torch.isclose(y3, _y3)

tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True]])